In [1]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog

#Create window for file selection
root = tk.Tk()

#Use file dialog to select the input file
file_path = filedialog.askopenfilename(title="Select the catalog to audit: ")

if not file_path:
    print("No file selected.")
    exit()

#Read the input file
df = pd.read_csv(file_path)

#Filter the dataframe based on a specific condition ('Track Item' == 'Y')
df_to_audit = df[df['Track Item'] == 'Y']

#Function to determine whether to sample 'Category' or 'Subcategory'
def determine_sampling_column(df_to_audit):
    if df_to_audit['Subcategory'].isnull().all():
        sampling_column = 'Subcategory'
    else:
        sampling_column = 'Category'
    return sampling_column

#Sub-function to perform the sampling based on the length of the catalog
def sample_items(df_to_audit, sampling_column, sample_fraction):
    if len(df_to_audit) == 0:
        return None
    else:
        return df_to_audit.groupby(sampling_column).apply(lambda x: x.sample(frac=sample_fraction, random_state=42) if len(x) * sample_fraction >= 2 else x.sample(n=2, random_state=42))

#Perform different levels of sampling based on catalog size
def simple_sample(df_to_audit):
    sampling_column = determine_sampling_column(df_to_audit)
    if len(df_to_audit) >= 15000:
        return sample_items(df_to_audit, sampling_column, 0.05)
    elif len(df_to_audit) >= 10000:
        return sample_items(df_to_audit, sampling_column, 0.03)
    elif len(df_to_audit) >= 1500:
        return sample_items(df_to_audit, sampling_column, 0.01)
    else:
        return None

#Results of the sampling
final_sample = simple_sample(df_to_audit)

#Mark items that need to be audited as "Audit"
df.loc[df['Retailer Item ID'].isin(final_sample['Retailer Item ID']), 'Track Item'] = 'Audit'

#Convert results to a CSV file
df.to_csv('Catalog_Audit.csv', index=False)

No file selected. Exiting.


FileNotFoundError: [Errno 2] No such file or directory: ''